In [ ]:
# Mini CNN test corrigé

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# --- 1. Transformations simples ---
transform = transforms.Compose([
    transforms.Resize((28,28)),
    transforms.ToTensor()
])

# --- 2. Dataset de test (MNIST) ---
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset  = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False)

# --- 3. Mini CNN corrigé ---
class MiniCNN(nn.Module):
    def __init__(self):
        super(MiniCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, 1)
        self.conv2 = nn.Conv2d(16, 32, 3, 1)
        self.fc1 = nn.Linear(32*5*5, 128)  # la taille exacte sera vérifiée par flatten
        self.fc2 = nn.Linear(128, 10)  # 10 classes MNIST

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = torch.flatten(x, 1)  # <-- aplati toutes les dimensions sauf batch
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# --- 4. Device, modèle, optimizer, loss ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MiniCNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# --- 5. Boucle d'entraînement simple (1 epoch) ---
model.train()
for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()

    if batch_idx % 100 == 0:
        print(f"Batch {batch_idx}, Loss: {loss.item():.4f}")

# --- 6. Test rapide ---
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        pred = output.argmax(dim=1)
        correct += (pred == target).sum().item()
        total += target.size(0)

print(f"Accuracy test mini CNN: {correct/total:.4f}")



In [ ]:

# Dataset & DataLoader

# 1️. Imports and device setup
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Force CPU (no GPU)
device = torch.device('cpu')
print("Using device:", device)

# ===============================
# 2️ Image Trafo


IMG_SIZE = 224
mean, std = [0.5], [0.5]  # normalization parameters

# Transformations for each split
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.RandomHorizontalFlip(p=0.5),  # random horizontal flip
        transforms.RandomRotation(5), # rotate ±5 degrees
        transforms.Grayscale(num_output_channels=1),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ]),
    'val': transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.Grayscale(num_output_channels=1),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ]),
    'test': transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.Grayscale(num_output_channels=1),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ]),
}


#================================0
# 3️ Datasets and DataLoaders

# Path to dataset in WSL
data_dir = "/home/chaouchix/data/chest_xray"

# Create datasets using ImageFolder
image_datasets = {
    split: datasets.ImageFolder(
        os.path.join(data_dir, split),
        transform=data_transforms[split]
    )
    for split in ['train', 'val', 'test']
}

# Create DataLoaders
dataloaders = {
    split: DataLoader(
        image_datasets[split], batch_size=32, shuffle=(split == 'train'),num_workers=2, pin_memory=False )
    for split in ['train', 'val', 'test']
}

# ===============================
# 4️. Check classes and dataset sizes


class_names = image_datasets['train'].classes
dataset_sizes = {k: len(v) for k, v in image_datasets.items()}

print("Classes:", class_names)
print("Dataset sizes:", dataset_sizes)

# ===============================
# 5️. Check a batch


x, y = next(iter(dataloaders['train']))
print("Batch images shape:", x.shape)
print("Batch labels shape:", y.shape)
print("Sample labels:", y[:10])


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Get a batch
x, y = next(iter(dataloaders['train']))

# Undo normalization for display
x = x * 0.5 + 0.5  # since normalize was (x-0.5)/0.5

# Plot first 4 images
fig, axes = plt.subplots(1, 4, figsize=(12,3))
for i in range(4):
    img = x[i][0].numpy()  # prendre le seul canal
    axes[i].imshow(img, cmap='gray')
    axes[i].set_title(class_names[y[i]])
    axes[i].axis('off')
plt.show()


In [ ]:
x, y = next(iter(dataloaders['train']))
print("Batch images shape:", x.shape)  # [batch_size, channels, H, W]
print("Batch labels shape:", y.shape)  # [batch_size]
print("Labels:", y[:10])
